In [1]:
import yaml
import requests
import time
from datetime import datetime, timedelta, date
import pandas as pd

Les données sont mises à disposition sur https://github.com/opencovid19-fr/data par sante-publique-france

## Automatisation de la récupération des données

In [2]:
r = requests.get('https://raw.githubusercontent.com/opencovid19-fr/data/master/sante-publique-france/2020-01-24.yaml')
r.status_code

200

In [54]:
r.text

'date: "2020-01-24"\nsource:\n  nom: Santé publique France\n  url: https://www.santepubliquefrance.fr/maladies-et-traumatismes/maladies-et-infections-respiratoires/infection-a-coronavirus/articles/cas-de-pneumonies-associees-a-un-nouveau-coronavirus-2019-ncov-a-wuhan-en-chine\n  archive: https://web.archive.org/web/20200124202351/https://www.santepubliquefrance.fr/maladies-et-traumatismes/maladies-et-infections-respiratoires/infection-a-coronavirus/articles/cas-de-pneumonies-associees-a-un-nouveau-coronavirus-2019-ncov-a-wuhan-en-chine\ndonneesNationales: # Voir archive 2 en bas de page (après le bulletin du 12/02)\n  casConfirmes: 3 # cas mise à jour suite à la publication du 27/01 de la SPF qui mentionnent ces 3 cas comme existant depuis le 24/01\n  hospitalises: 3\n  deces: 0\ndonneesRegionales: # Voir archive 2 en bas de page (après le bulletin du 12/02)\n  - nom: Île-de-France\n    code: REG-11\n    casConfirmes: 2\n    hospitalises: 2\n  - nom: Nouvelle-Aquitaine\n    code: REG-7

In [55]:
y = yaml.load(r.text, Loader=yaml.FullLoader)

In [56]:
y

{'date': '2020-01-24',
 'source': {'nom': 'Santé publique France',
  'url': 'https://www.santepubliquefrance.fr/maladies-et-traumatismes/maladies-et-infections-respiratoires/infection-a-coronavirus/articles/cas-de-pneumonies-associees-a-un-nouveau-coronavirus-2019-ncov-a-wuhan-en-chine',
  'archive': 'https://web.archive.org/web/20200124202351/https://www.santepubliquefrance.fr/maladies-et-traumatismes/maladies-et-infections-respiratoires/infection-a-coronavirus/articles/cas-de-pneumonies-associees-a-un-nouveau-coronavirus-2019-ncov-a-wuhan-en-chine'},
 'donneesNationales': {'casConfirmes': 3, 'hospitalises': 3, 'deces': 0},
 'donneesRegionales': [{'nom': 'Île-de-France',
   'code': 'REG-11',
   'casConfirmes': 2,
   'hospitalises': 2},
  {'nom': 'Nouvelle-Aquitaine',
   'code': 'REG-75',
   'casConfirmes': 1,
   'hospitalises': 1,
   'donneesRegionales': None}],
 'donneesDepartementales': [{'nom': 'Gironde',
   'code': 'DEP-33',
   'casConfirmes': 1,
   'hospitalises': 1}],
 'donneesM

### Avec une seule fonction ...

In [57]:
r = requests.get('https://raw.githubusercontent.com/opencovid19-fr/data/master/sante-publique-france/2020-03-24.yaml')
if (r.status_code==200):
    print (yaml.load(r.text, Loader=yaml.FullLoader))

{'date': datetime.date(2020, 3, 24), 'source': {'nom': 'Santé publique France', 'url': 'https://www.santepubliquefrance.fr/maladies-et-traumatismes/maladies-et-infections-respiratoires/infection-a-coronavirus/articles/infection-au-nouveau-coronavirus-sars-cov-2-covid-19-france-et-monde', 'archive': 'https://web.archive.org/web/20200324193012/https://www.santepubliquefrance.fr/maladies-et-traumatismes/maladies-et-infections-respiratoires/infection-a-coronavirus/articles/infection-au-nouveau-coronavirus-sars-cov-2-covid-19-france-et-monde'}, 'donneesNationales': {'casConfirmes': 22302, 'deces': 1100}, 'donneesRegionales': [{'nom': 'Auvergne-Rhône-Alpes', 'code': 'REG-84', 'casConfirmes': 1857}, {'nom': 'Bourgogne-Franche-Comté', 'code': 'REG-27', 'casConfirmes': 1510}, {'nom': 'Bretagne', 'code': 'REG-53', 'casConfirmes': 526}, {'nom': 'Centre-Val de Loire', 'code': 'REG-24', 'casConfirmes': 450}, {'nom': 'Corse', 'code': 'REG-94', 'casConfirmes': 217}, {'nom': 'Grand Est', 'code': 'REG-

In [59]:
hier = date.today() + timedelta(-1)
shier = hier.strftime("%Y-%m-%d")
fichier = 'https://raw.githubusercontent.com/opencovid19-fr/data/master/sante-publique-france/' + shier + '.yaml'

## Création d'une fonction qui récupère les données d'hier sur le Covid19

In [60]:
def RecupCovid19Hier():
    hier = date.today() + timedelta(-1)
    shier = hier.strftime("%Y-%m-%d")
    fichier = 'https://raw.githubusercontent.com/opencovid19-fr/data/master/sante-publique-france/' + shier + '.yaml'
    out = ""
    r = requests.get(fichier)
    if (r.status_code==200):
       out = yaml.load(r.text, Loader=yaml.FullLoader)
    return out

In [61]:
jsonout = RecupCovid19Hier()
jsonout

''

In [62]:
jsonout['date'].strftime("%Y-%m-%d")

TypeError: string indices must be integers

In [34]:
jsonout['donneesNationales']['casConfirmes']

22302

## Sérialisation des données dans un dataframe

In [61]:
A = pd.DataFrame()
A = A.append({
                'Date' : jsonout['date'].strftime("%Y-%m-%d") , 
                'FR Cas Confirmés' : jsonout['donneesNationales']['casConfirmes'] , 
                'FR Décès' : jsonout['donneesNationales']['deces'], 
                'WW Cas Confirmés' : jsonout['donneesMondiales']['casConfirmes'],
                'WW Décès' : jsonout['donneesMondiales']['deces']
             } , 
             ignore_index=True)
A

,Date,FR Cas Confirmés,FR Décès,WW Cas Confirmés,WW Décès
0,2020-03-24,22302.0,1100.0,378041.0,16365.0


In [75]:
datetime_object = datetime.strptime(StartingDate, '%Y-%m-%d')
datetime_object

datetime.datetime(2020, 1, 24, 0, 0)

In [79]:
date_str = "2020-01-24"
date_object = datetime.strptime(date_str, '%Y-%m-%d').date()
print(date_object)

2020-01-24
